# Spatial analysis

After running PIPEX on our images, we would like to load the images and results into a `SpatialData` Object. Afterwards, we can interactively look at it with Napari  or run more spatial analyses with SquidPy

In [1]:
from anndata import read_h5ad, AnnData
import pandas as pd
from pathlib import Path
from napari_spatialdata import Interactive
import numpy as np
from spatialdata import SpatialData
from spatialdata.models import Image2DModel, Labels2DModel, TableModel
import tifffile

## Data download

You could use the data generated during the course while running PIPEX. Or you might also download the data from [here](https://ell-vault.stanford.edu/dav/fredbn/www/I2K/pipex_data.zip) and unzip it into the data folder inside the `I2K2024-MTIWOKSHOP` folder.

If it didn't work, you can uncomment the lines in the cell below and run them.

In [ ]:
# from utils import download_and_unzip

# file_url = "https://ell-vault.stanford.edu/dav/fredbn/www/I2K/pipex_data.zip"
# output_folder = '../data'
# download_and_unzip(file_url, output_folder)

## SpatialData construction

We can begin by stating the directories of the pipex results, images and where we would like to store the SPatialData Objects as a Zarr.

In [7]:
pipex_dir = Path("../data/data/analysis")
images_dir = Path("../data/data")
sdata_file = Path("../data/pipex.zarr")

In [ ]:
channels = []
channel_imgs = []
for channel_img_file in sorted(images_dir.glob("*.tif")):
    channel_img = tifffile.imread(channel_img_file)
    channels.append(channel_img_file.stem)
    channel_imgs.append(channel_img)
img = np.stack(channel_imgs)
img.shape

In [ ]:
labels_file = pipex_dir / "segmentation_data.npy"
labels = np.load(labels_file).astype(np.uint16)
labels.shape

In [ ]:
cells_file = pipex_dir / "downstream" / "anndata_TissUUmaps.h5ad"
cells = read_h5ad(cells_file) 
cells

We need to add some information to relate the AnnData information in the table to the cells detected and shown in the labels.

In [11]:
cells.uns["spatialdata_attrs"] = {"region": ["labels"],
                                  "region_key": "sample_id",
                                  "instance_key": "id"}
cells.obs["sample_id"] = "labels"
cells.obs["sample_id"] = cells.obs["sample_id"].astype("category")

In [ ]:
sdata = SpatialData(
    images={"images": Image2DModel.parse(img, dims=("c", "y", "x"), c_coords=channels)},
    labels={"labels": Labels2DModel.parse(labels, dims=("y", "x"))},
    tables={"table": TableModel.parse(cells)},
)
sdata.write(sdata_file, overwrite=True)
sdata

In [ ]:
Interactive(sdata)

## Spatial single-cell analysis

We can begin by reading the spatial data file constructed on the previous subsection.

In [ ]:
sdata_file = "../data/pipex.zarr"
sdata = SpatialData.read(sdata_file)
sdata

We can import squidpy and scanpy to perform analysis on our files, but most of these analyses have already been performed and are saved in the appended tables.

In [15]:
# import scanpy as sc
import squidpy as sq

In [ ]:
sdata["table"]

In [ ]:
sdata["table"].var_names, sdata["table"].obs_names

We can visualize the lolcation of every cluster in the plot with the following function.

In [ ]:
sq.pl.spatial_scatter(sdata["table"], shape=None, color="leiden", size=50)

For the following, we will do some of the steps presented in [this link](https://www.sc-best-practices.org/spatial/neighborhood.html#). We can compute a spatial graph by running `spatial_neighbours` function in the `gr` module (graph). And then we can run `spatial_scatter` once more from the `pl` (plot) module, adding the `connectivity_key` parameter.

*Note*: Feel free to play around with the radius parameter and see what happens when you increase it to 100.

In [ ]:
sq.gr.spatial_neighbors(sdata["table"], radius=30.0, coord_type="generic")
sq.pl.spatial_scatter(
    sdata["table"],
    color="leiden",
    connectivity_key="spatial_connectivities",
    edges_color="black",
    shape=None,
    edges_width=1,
    size=30,
)

We could as well rerun and plot (or only plot) the neighbourhood enrichment test.

In [ ]:
# sq.gr.spatial_neighbors(sdata["cells"])
# sq.gr.nhood_enrichment(sdata["cells"], cluster_key="leiden")
sq.pl.nhood_enrichment(sdata["table"], cluster_key="leiden", figsize=(5, 5))

We can calculate the interaction matrix which is the sum of all interactions happenning in the tissue.

In [21]:
sq.gr.interaction_matrix(sdata["table"], cluster_key="leiden")

In [ ]:
sq.pl.interaction_matrix(sdata["table"], cluster_key="leiden", method="average", figsize=(5, 5))

We could overlay the graphs with the an image to see how it looks like.

*Note*: This part is under development and might work for spatial transcriptomics with spots, such as visium, but it was not working with our dataset.

In [56]:
spatial_key = "spatial"
library_id = "labels"
sdata["cells"] = sdata["table"]

sdata["cells"].uns[spatial_key] = {library_id: {}}
sdata["cells"].uns[spatial_key][library_id]["images"] = {}
sdata["cells"].uns[spatial_key][library_id]["images"] = {"hires": sdata["images"].loc["DAPI"].to_numpy()}
sdata["cells"].uns[spatial_key][library_id]["scalefactors"] = {
    "tissue_hires_scalef": 1,
    "spot_diameter_fullres": 0.5,
}

In [ ]:
sdata["cells"].uns['spatial']

In [ ]:
sq.pl.spatial_scatter(sdata["cells"], color="leiden")

## Generating SpatialData Object from csv file

In case wyou have some extra time (or extra curiosity), we could go one step back and load the csv file into SpatialData object. Once this is done, we could run our own dimensionality reduction and clustering algorithms.

In [38]:
pipex_dir = Path("../data/data/analysis")
images_dir = Path("../data/data")
sdata_file = Path("../data/pipex.zarr")

In [ ]:
channels = []
channel_imgs = []
for channel_img_file in sorted(images_dir.glob("*.tif")):
    channel_img = tifffile.imread(channel_img_file)
    channels.append(channel_img_file.stem)
    channel_imgs.append(channel_img)
img = np.stack(channel_imgs)
img.shape

In [ ]:
labels_file = pipex_dir / "segmentation_data.npy"
labels = np.load(labels_file).astype(np.uint16)
labels.shape

In [ ]:
cells_file = pipex_dir /  "cell_data_ANNOTATED.csv"
cells = pd.read_csv(cells_file)
cells.kmeans_annotated.fillna("None", inplace=True)
cells

In [ ]:
markers = ["ARL13B", "ATM", "Bcl-2", "Beta-actin", "CD107a", "CD11c", "CD14", "CD163", 
           "CD20", "CD31", "CD34", "CD39", "CD3e", "CD4", "CD40", "CD44", "CD45", "CD56", 
           "CD68", "CD8", "Caveolin", "Collagen_IV", "DAPI", "E-cadherin", "ER", "EpCam", 
           "GP100", "HIF1A", "HLA-A", "HLA-DR", "Histone_H3_pSer28", "ICOS", "IDO1", "Keratin_5",
             "Keratin_8-18", "Ki67", "PCNA", "PCNT", "PD-1", "PD-L1", "Pan-Cytokeratin", "Podoplanin", 
             "SMA", "SOX2", "TOX", "VISTA", "Vimentin", "b-Catenin1", "iNOS"]

# Create core of AnnData object
adata = AnnData(X = cells[markers].values,
                obs = cells.rename(columns={"cell_id": "id"})[["id"]],
                var = pd.DataFrame(index=markers))

# Add layers for variables
for statistical_descriptor in ['local_90', 'ratio_pixels', 'bin']:
    adata.layers[statistical_descriptor] = cells[["_".join([marker, statistical_descriptor]) for marker in markers]].values

# Add observations
# Note: if you want to redo the leiden or kmeans clustering, you might want to skip adding them
for col in ["size", "x", "y", "leiden",	"leiden_color",	"kmeans",	"kmeans_color",	"kmeans_annotated",	"kmeans_annotated_color"]:
  adata.obs[col] = cells[col].values

adata

We need to add some information to relate the AnnData information in the table to the cells detected and shown in the labels.

In [58]:
adata.uns["spatialdata_attrs"] = {"region": ["labels"],
                                  "region_key": "sample_id",
                                  "instance_key": "id"}
adata.obs["sample_id"] = "labels"
adata.obs["sample_id"] = adata.obs["sample_id"].astype("category")

In [ ]:
sdata = SpatialData(
    images={"images": Image2DModel.parse(img, dims=("c", "y", "x"), c_coords=channels)},
    labels={"labels": Labels2DModel.parse(labels, dims=("y", "x"))},
    tables={"table": TableModel.parse(adata)},
)
sdata.write(sdata_file, overwrite=True)
sdata

In [ ]:
Interactive(sdata)

For clustering, we could use scanpy over the AnnData object

In [ ]:
import scanpy as sc

sc.pp.normalize_total(sdata["cells"])
sc.pp.log1p(sdata["cells"])
sc.pp.pca(sdata["cells"])
sc.pp.neighbors(sdata["cells"])
sc.tl.umap(sdata["cells"])
sc.tl.leiden(sdata["cells"])

After this step, we would reanalyze and maybe annotate again the clusters found.